In [230]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

url = "https://github.com/amitaslanov/final_project/raw/main/output_all_students_Train_v10.xlsx"

# Read the Excel file from the URL
xls = pd.ExcelFile(url)

# Parse the first sheet of the Excel file
df = xls.parse(0)

#### Part 2 - Data Prep

In [231]:
# "City" column
df.loc[:, 'City'] = df['City'].str.replace(' נהריה', 'נהריה')
df.loc[:, 'City'] = df['City'].str.replace('נהריה', 'נהרייה')
df.loc[:, 'City'] = df['City'].str.replace('שוהם', ' שוהם')

In [232]:
# Clean the "price" column
df.loc[:, 'price'] = df['price'].astype(str).str.replace('[^\d.]', '', regex=True)
df.loc[:, 'price'] = pd.to_numeric(df['price'])

# Drop rows with missing prices
df.dropna(subset=['price'], inplace=True)

In [233]:
# Clean the "room_number" column
df.loc[:, 'room_number'] = df['room_number'].astype(str).str.replace('[^\d.]', '', regex=True)
df.loc[:, 'room_number'] = pd.to_numeric(df['room_number'])

rows_to_drop = df[df['room_number'] > 10].index

# Drop the identified rows
df = df.drop(rows_to_drop, axis=0)

In [234]:
# "Area" column
df.loc[:, 'Area'] = df['Area'].astype(str).str.replace('[^\d.]', '', regex=True)
df.loc[:, 'Area'] = pd.to_numeric(df['Area'])

In [235]:
# "Street" column
df.loc[:, 'Street'] = df['Street'].str.replace(r"[\[\]']", '', regex=True)

In [236]:
# "number_in_street" column
df.loc[:, 'number_in_street'] = pd.to_numeric(df['number_in_street'], errors='coerce')

In [237]:
# "type" column
df.loc[:, 'type'] = df['type'].str.replace("'", "")

In [238]:
# "city_area" column 
df.loc[:, 'city_area'] = df['city_area'].astype(str)
df.loc[:, 'city_area'] = df['city_area'].apply(lambda x: '' if not re.match(r'^[\u0590-\u05FF\s-]+$', x) else x)
df.loc[:, 'city_area'] = df['city_area'].str.replace('-', '')

In [239]:
# "floor_out_of" column
df.loc[:, 'floor_out_of'] = df['floor_out_of'].str.replace('קומה', '')
df.loc[:, 'floor_out_of'] = df['floor_out_of'].str.replace('קומת ', '')
df.loc[:, 'floor_out_of'] = df['floor_out_of'].str.replace('מתוך', ' ')
df.loc[:, 'floor_out_of'] = df['floor_out_of'].str.replace('מרתף', '-1')
df.loc[:, 'floor_out_of'] = df['floor_out_of'].str.replace('קרקע', '0')
df.loc[:, 'floor_out_of'] = df['floor_out_of'].str.replace('\s+', ' ')

C:\Users\amits\AppData\Local\Temp\ipykernel_22516\2592325436.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'floor_out_of'] = df['floor_out_of'].str.replace('\s+', ' ')


In [240]:
# "total_floors" and "floor_out_of" columns
df[['floor', 'total_floors']] = df['floor_out_of'].str.extract(r'(\d+)\D*(\d*)')
df['floor'] = df.apply(lambda row: row['total_floors'] if pd.isnull(row['floor']) else row['floor'], axis=1)
df['total_floors'] = df.apply(lambda row: row['floor'] if pd.isnull(row['total_floors']) else row['total_floors'], axis=1)
df = df.drop('floor_out_of', axis=1)

# Define the list of types that should result in None for "floor" and "total_floors"
types_to_ignore = ["בית פרטי", "דו משפחתי", "מגרש", "נחלה", "קוטג'", "קוטג' טורי"]

# Update "floor" and "total_floors" to None for the specified types
df.loc[df['type'].isin(types_to_ignore), ['floor', 'total_floors']] = 'None'

In [241]:
# "has ___" columns
columns_to_convert = ['hasElevator ', 'hasParking ', 'hasBars ','hasStorage ','hasAirCondition ','hasBalcony ','hasMamad ','handicapFriendly ']
for column in columns_to_convert:
    df[column] = df[column].apply(lambda x: 0 if pd.isna(x) else (1 if x in [True, 'true'] else (0 if x in [False, 'false', 'אין', 'לא', 'no'] else (1 if any(word in str(x) for word in ['יש', 'כן', 'yes']) else 0))))

In [242]:
# "entranceDate" column
df['entranceDate '] = df['entranceDate '].replace({
    'גמיש': 'flexible',
    'גמיש ': 'flexible',
    'לא צויין': 'not_defined',
    'מיידי': 'immediate'
})


df['days'] = ""

date_format = "%Y-%m-%d"  

for index, row in df.iterrows():
    entrance_date = row['entranceDate ']
    
    if isinstance(entrance_date, datetime):
        formatted_date = entrance_date.strftime(date_format)
        
        days = (entrance_date - datetime.now()).days
        df.at[index, 'entranceDate '] = formatted_date
        df.at[index, 'days'] = days
        
        if days < 180 or df.at[index, 'entranceDate '] == 'immediate':
            df.at[index, 'entranceDate '] = "less_than_6 months"
        elif 180 <= days <= 365:
            df.at[index, 'entranceDate '] = "months_6_12"
        else:
            df.at[index, 'entranceDate '] = "above_year"
            
df = df.drop('days', axis=1)

In [243]:
# "condition" column
df['condition '] = df['condition '].str.replace('FALSE', 'None')
df['condition '] = df['condition '].str.replace('לא צויין', 'None')
df['condition '].fillna('None', inplace=True)

In [244]:
# "publishedDays" column
df['publishedDays '] = df['publishedDays '].apply(lambda x: re.sub('[^\d]+', '', str(x)) if pd.notnull(x) else '')
df['publishedDays '] = pd.to_numeric(df['publishedDays '], errors='coerce')

In [245]:
#droping rows that the price is less then 500k (probably a input error)
df.drop(df[df['price'] < 500000].index, inplace=True)

In [246]:
#fill in missing values
df['floor'] = pd.to_numeric(df['floor'], errors='coerce')
df['total_floors'] = pd.to_numeric(df['total_floors'], errors='coerce')
df['floor'].fillna(0, inplace=True)
df['total_floors'].fillna(0, inplace=True)

df['num_of_images'].fillna(0, inplace=True)

mean_value_room_number = df['room_number'].mean()
df['room_number'].fillna(mean_value_room_number, inplace=True)

mean_value_area = df['Area'].mean()
df['Area'].fillna(mean_value_area, inplace=True)

mean_value_number_in_street = df['number_in_street'].mean()
df['number_in_street'].fillna(mean_value_number_in_street, inplace=True)

mean_value_publishedDays = df['publishedDays '].mean()
df['publishedDays '].fillna(mean_value_publishedDays, inplace=True)

In [247]:
df = df.drop(['Street', 'city_area', 'description '], axis=1)

##### We decided to drop the street and city_area columns because there is alot of different streests and area in every city, and if we will do one-hot-encoding to these column we will get a huge data.
##### From our knowledge these details has a very minor effect on the price.

##### We decided to drop the details column because it is not categorial and the important details the written in it provided in the othe columns (rooms/area and more).

In [255]:
# Check for duplicate rows
duplicate_rows = df.duplicated()

# Remove duplicate rows
df = df.drop_duplicates()

# Optional: Reset the index of the DataFrame
df = df.reset_index(drop=True)

#### one hot encoding

In [256]:
# Columns to perform one-hot encoding on
columns_to_encode = ['City', 'type', 'condition ', 'entranceDate ', 'furniture ']

# Perform one-hot encoding
encoded_df = pd.get_dummies(df, columns=columns_to_encode)

In [257]:
encoded_df.columns = encoded_df.columns.str.replace(' ', '')

#### features selection

In [258]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# Assuming features_selection_data is your feature matrix and 'price' is the target variable
X = encoded_df.drop('price', axis=1)
y = encoded_df['price']

# Create the linear regression model (or choose a different model)
model = LinearRegression()

# Create the RFE object with automatic feature selection
rfe = RFE(estimator=model, n_features_to_select=None)

# Fit the RFE model to the data
rfe.fit(X, y)

# Get the selected feature indices
selected_indices = rfe.support_

# Get the selected feature names
selected_features = X.columns[selected_indices]

# Print the selected feature names
print("Selected Features:")
for feature in selected_features:
    print(feature)

Selected Features:
hasElevator
hasBars
hasStorage
hasAirCondition
hasBalcony
handicapFriendly
City_שוהם
City_אילת
City_ביתשאן
City_בתים
City_גבעתשמואל
City_חולון
City_ירושלים
City_נהרייה
City_נתניה
City_צפת
City_קריתביאליק
City_רחובות
City_תלאביב
type_אחר
type_ביתפרטי
type_בניין
type_דומשפחתי
type_דופלקס
type_דירה
type_דירתגג
type_דירתגן
type_טריפלקס
type_מיניפנטהאוז
type_נחלה
type_קוטגטורי
condition_None
condition_דורששיפוץ
condition_חדש
condition_ישן
condition_משופץ
condition_שמור
furniture_אין


###### We noticed that the room_number and Area columns wasnt chosen and we know these features can highly effect the property price so we will add them.
###### We also noticed there is only 1 options of furniture (out of 4) so we will drop it.
###### So we will check the correlation between these two features and the price.

In [259]:
selected_features = selected_features.tolist()
selected_features.append('room_number')
selected_features.append('Area')
selected_features.remove('furniture_אין')
selected_features = pd.Index(selected_features)

In [260]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming selected_features contains the list of selected feature names
X = encoded_df[selected_features]
y = encoded_df['price']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit the Elastic Net model
model = ElasticNet(alpha=0.5, l1_ratio=0.5)  # Adjust alpha and l1_ratio as needed
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 3248272135008.5693


In [261]:
from sklearn.linear_model import ElasticNet
import joblib

# Assuming selected_features contains the list of selected feature names
X = encoded_df[selected_features]
y = encoded_df['price']

# Create and fit the Elastic Net model
model = ElasticNet(alpha=0.5, l1_ratio=0.5)  # Adjust alpha and l1_ratio as needed
model.fit(X, y)

# Save the trained model as a PKL file
joblib.dump(model, 'trained_model.pkl')

['trained_model.pkl']